In [167]:
import requests
from bs4 import BeautifulSoup as bs
import zipfile
from urllib import request as req
import bs4.element as elem
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
import collections
import re
# import pandas as pd

# 모든 회사의 정보

In [168]:
url = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=f34638c11aaa3ed9ee3fbbabec9b0a9762a4a628'
downfile = 'corp_code.zip'
req.urlretrieve(url, downfile)
xml = ''
corp_list = []
with zipfile.ZipFile(downfile) as z:
    z.extractall()
    xml = z.filelist[0].filename
with open(xml,'r', encoding='utf-8') as f:
    soup = bs(f.read())
    corp_list = soup.select('list')

In [171]:
corp_name = input('찾고자 하는 종목 입력 : ')
corp_code = ''
for item in corp_list:
    if item.select_one('corp_name').text == corp_name:
        corp_code = item.select_one('corp_code').text
print(corp_name, corp_code)

찾고자 하는 종목 입력 : 삼성바이오로직스
삼성바이오로직스 00877059


# 분기보고서 찾기 

In [172]:
url = 'https://opendart.fss.or.kr/api/list.json'
params = {
    'crtfc_key': 'f34638c11aaa3ed9ee3fbbabec9b0a9762a4a628',
    'bgn_de': '20200101',
    'corp_code': corp_code,
    'pblntf_ty': 'A'
}
resp = requests.get(url, params=params)
report_list = resp.json()['list']
rcept_no = ''
for report in report_list:
    if report['report_nm'].split(' ')[0] == '분기보고서':
        rcept_no = report['rcept_no']

# 분기보고서 데이터 가져오기

In [173]:
url = 'https://opendart.fss.or.kr/api/document.xml'
url += '?crtfc_key=f34638c11aaa3ed9ee3fbbabec9b0a9762a4a628'
url += f'&rcept_no={rcept_no}'
downfile = 'quarter_rep.zip'
req.urlretrieve(url, downfile)

xml = ''
elem = elem
with zipfile.ZipFile(downfile) as z:
    z.extractall()
    xml = z.filelist[0].filename
with open(xml,'r') as f:
    soup = bs(f.read())
    elem = soup.select_one('title[aassocnote="D-0-2-0-0"]')

('quarter_rep.zip', <http.client.HTTPMessage at 0x22c361605c8>)

In [179]:
target = elem.parent.text.replace('&cr;','').replace('\n','')
hannaum = Hannanum()
token = hannaum.morphs(target)
counter = collections.Counter(token)
print(counter)

Counter({'.': 193, '하': 156, ',': 149, 'ㄴ': 141, '는': 133, '의': 131, '에': 129, '이': 128, '을': 117, '및': 78, '어': 76, ')': 74, '를': 72, '은': 66, '습니다': 65, '있': 64, '되': 64, '등': 55, '(': 55, '당사': 48, '가': 45, '으로': 43, 'ㅂ니다': 42, '고': 37, '과': 36, '하고': 35, '생산': 33, '-': 32, 'CMO': 27, 'ㄹ': 26, '대하': 24, ':': 24, '아': 21, '위하': 19, '같': 19, '제품': 18, '/': 17, '며': 17, '로': 17, '생산설비': 17, '와': 17, '바이오의약품': 16, '의약품': 16, '통하': 16, '사업': 15, '으며': 15, '에서': 15, '따르': 15, '관하': 15, '수': 15, '중': 14, '구': 14, '개발': 13, '었습니다': 13, '기준': 13, '지': 13, '도': 13, '관련': 13, '단위': 13, '연구개발': 12, '현황': 12, '시장': 12, '다음': 12, '게': 12, '고객사': 12, '주요': 12, '%': 11, '온실가스': 11, '글로벌': 10, '기': 10, '2': 10, '없': 10, '서비스': 10, ')-': 10, 'CDO': 9, '현재': 9, '확대': 9, '대': 9, '나': 9, '받': 9, '원재료': 9, '관리': 9, '체결': 9, "'": 9, '법률': 9, '세포주': 8, '고객': 8, '것': 8, '과정': 8, 'ㅁ': 8, '계약': 8, '해당': 8, '위험': 8, '천원': 8, '공정': 7, '1': 7, '주': 7, '시장규모': 7, '보유': 7, '5': 7, '또는': 7, '사항': 7, '적': 7, '조건': 7

In [183]:
def del_pattern_dict(pattern, target_dict):
    key_list = []
    for key in target_dict.keys():
        if pattern.match(key):
            key_list.append(key)
    for i in key_list:
        del target_dict[i]

In [184]:
gae = re.compile('[0-9]+개$')
jaeum = re.compile('[ㄱ-ㅎ]')
sp = re.compile('^[^\w\s]+$')
del_pattern_dict(gae, counter)
del_pattern_dict(jaeum, counter)
del_pattern_dict(sp, counter)

In [185]:
stop_list = []
with open('stop_word.txt','r',encoding='utf-8') as f:
    stop_list = f.read().split('\n')

In [190]:
# commons
for stop in stop_list:
    del counter[stop]
# custroms
del counter['II']
del counter['사업']
del counter['개요']
del counter['현황당사']
del counter['사업부문별']
del counter['본사']
del counter['당사']
del counter['산하']
del counter['한국']
del counter['부문']
del counter['내용◆click◆『수주상황』']
del counter['삽입11013#*_수주상황']


In [191]:
counter.most_common()

[('은', 66),
 ('고', 37),
 ('생산', 33),
 ('CMO', 27),
 ('대하', 24),
 ('위하', 19),
 ('같', 19),
 ('제품', 18),
 ('며', 17),
 ('생산설비', 17),
 ('바이오의약품', 16),
 ('의약품', 16),
 ('통하', 16),
 ('으며', 15),
 ('따르', 15),
 ('관하', 15),
 ('수', 15),
 ('중', 14),
 ('개발', 13),
 ('기준', 13),
 ('관련', 13),
 ('단위', 13),
 ('연구개발', 12),
 ('현황', 12),
 ('시장', 12),
 ('게', 12),
 ('고객사', 12),
 ('주요', 12),
 ('온실가스', 11),
 ('글로벌', 10),
 ('2', 10),
 ('없', 10),
 ('서비스', 10),
 ('CDO', 9),
 ('현재', 9),
 ('확대', 9),
 ('대', 9),
 ('받', 9),
 ('원재료', 9),
 ('관리', 9),
 ('체결', 9),
 ('법률', 9),
 ('세포주', 8),
 ('고객', 8),
 ('과정', 8),
 ('계약', 8),
 ('해당', 8),
 ('위험', 8),
 ('천원', 8),
 ('공정', 7),
 ('1', 7),
 ('주', 7),
 ('시장규모', 7),
 ('보유', 7),
 ('5', 7),
 ('또는', 7),
 ('사항', 7),
 ('적', 7),
 ('조건', 7),
 ('환위험', 7),
 ('규제', 7),
 ('바', 6),
 ('진행', 6),
 ('전체', 6),
 ('품질관리', 6),
 ('역량', 6),
 ('매출', 6),
 ('이러하', 6),
 ('4', 6),
 ('기존', 6),
 ('2018년', 6),
 ('가동', 6),
 ('거나', 6),
 ('of', 6),
 ('고객사의', 6),
 ('않', 6),
 ('and', 6),
 ('연구', 6),
 ('배출', 6),
 ('CRO'

# 기준
## 언급 횟수가 많을수록 키워드
## 다른 종목에서 언급이 안됐을수록 키워드
## 특수문자가 포함되면 키워드가 아님